# Example processing script for GOES/ABIdata
### A simple example for retrieving sea surface reflectance using data from the GOES/ABI instrument.

This script uses free and open GOES/ABI data from NOAA's big data project, you do not need to download data in advance as
this is done within the notebook itself.

Output from this example notebook is available on zenodo: https://doi.org/10.5281/zenodo.7885995

NOTE: Additional libraries are required for this notebook:
 1) To enable the remote data to be read: `s3fs`.
 2) For loading the data in satpy: `h5netcdf`.
 3) For saving the results: `rasterio`.

Here we compute the sea surface reflectance based on assumed constant winds across the globe. We do not apply cloud or land masking and we do not calculate any of the BRDF parameters.

This example assumes that output data should be saved to an out subfolder. This may not be convenient, so you can change the directory using the `out_dir` variable below.

In [6]:
%load_ext autoreload
%autoreload 2

from satpy import Scene
from pycoxmunk import PyCoxMunk
import numpy as np
import s3fs
import os

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# Set which bands we wish to load
bnames = ['C01', 'C02', 'C04']

# Set the directory containing the SLSTR data dir
# This folder should contain one or more subfolders with the SLSTR data (ending .SEN3)
# To save memory and CPU time, only use a few granules maximum for this.
# Downloaded GOES data will be placed into this directory.
#
# NOTE: Downloaded data is not automatically deleted after the notebook is run!
# You must manually delete the files from this directory.
indir = './'

# Set the directory where we'll save the output
outdir = './out/'

In [27]:
# Preparation, download the data
os.makedirs(indir, exist_ok=True)
os.makedirs(outdir, exist_ok=True)

# Set up an S3 filesystem to download the data
fs = s3fs.S3FileSystem(anon=True)

# Loop over the bands we have chosen, construct filenames and download if needed.
# NOTE: This may take some time depending on which bands you selected and the speed
# of your internet connection. the default bands (1, 2, 4) require ~560Mb download.

final_fnames = []
for band in bnames:
    cur_fname = fs.glob(f's3://noaa-goes18/ABI-L1b-RadF/2023/030/20/*{band}*_s20230302000*')[0]
    destination_fname = os.path.join(indir, cur_fname.split('/')[-1])
    if not os.path.exists(destination_fname):
        fs.get_file(cur_fname, destination_fname)
    final_fnames.append(destination_fname)

In [34]:
# Create a scene by searching for files
scn = Scene(final_fnames, reader='abi_l1b')
# Load the bands
scn.load(bnames)

# GOES / ABI has bands of different resolutions. PyCoxMunk requires all data to be at the same resolutuion, so we must resample.
# Here we resample to the coarsest resolution, 1km at nadir.
# We use the `native` resampler as it is the fastest, but other options such as `bilinear` produce more pleasing results.
scn = scn.resample(scn.coarsest_area(), resampler='native')

In [35]:
# Create the PyCoxMunk class for processing
# The SLSTR data contains angle information, so here we specify `angle_names=None` to prevent
# PyCoxMunk from computing angles internally.
pcm = PyCoxMunk(scn, bnames)

In [36]:
# Here we assume a uniform wind, needed for cases we don't have actual wind data
# See SEVIRI example for case where wind data is available
pcm.setup_wind(1., 1.)

In [37]:
# Retrieve the sea surface reflectance
pcm.retr_coxmunk_refl()

In [38]:
# Save results to the directory specified earlier
# Each dataset is saved to its own file.
pcm.scn.save_datasets(base_dir=outdir, enhance=False, dtype=np.float32)